# Setup

In [ ]:
!./init_s3_bucket.sh

In [ ]:
!./init_schema.sh

In [ ]:
!pip install sagemaker-studio-image-build

In [ ]:
!sm-docker build docker/. --repository algotrading:1.0

# Attach Container as SageMaker Kernel

In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [ ]:
!aws --region {region} sagemaker delete-image-version --image-name algotrading --version-number=1
!aws --region {region} sagemaker list-image-versions --image-name algotrading
!aws --region {region} sagemaker delete-image --image-name algotrading
!aws --region {region} sagemaker list-images
!aws --region {region} sagemaker delete-app-image-config --app-image-config-name=algotrading-config
!aws --region {region} sagemaker list-app-image-configs

In [ ]:
!aws --region {region} sagemaker create-image --image-name algotrading --role-arn {role}
!aws --region {region} sagemaker list-images
!aws --region {region} sagemaker create-image-version --image-name algotrading --base-image "{account}.dkr.ecr.{region}.amazonaws.com/algotrading:1.0"
!aws --region {region} sagemaker list-image-versions --image-name algotrading
!aws --region {region} sagemaker create-app-image-config --cli-input-json file://app-image-config-input.json
!aws --region {region} sagemaker list-app-image-configs

In [ ]:
domainid=!(aws --region {region} sagemaker list-domains --query 'Domains[0].DomainId' --output text)
domainid=domainid[0]
domainid

In [ ]:
!aws --region {region} sagemaker update-domain --domain-id {domainid} --cli-input-json file://default-user-settings.json